# Connect to redshift

In [ ]:
import psycopg2
import boto3

# Redshift connection parameters
host = '<Your_redshift-cluster_host>'
port = '5439'
dbname = 'dev'
user = 'admin'
password = 'Password123'

# Establish connection to Redshift
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    
    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    # Example SQL query
    sql_query = "SELECT COUNT(*) FROM public.staff_raw;"

    # Execute the query
    cursor.execute(sql_query)

    # Fetch the result
    result = cursor.fetchall()

    # Print the results
    for row in result:
        print(row)

except Exception as e:
    print(f"Error: {e}")


(1325324,)


In [5]:
sql_query = '''
    SELECT DISTINCT Role FROM public.staff_hours;


'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('rndon',)
('rn',)
('lpn',)
('natrn',)
('rnadmin',)
('lpnadmin',)
('cna',)
('medaide',)


# Analysis

## 1. Analysis by Region

### 1.1 Nurse Staffing ratio by State

In [2]:
sql_query = '''
    SELECT 
        STATE,
        SUM(Hrs_RNDON +
            Hrs_RNadmin +
            Hrs_RN +
            Hrs_LPNadmin +
            Hrs_LPN +
            Hrs_CNA +
            Hrs_NAtrn +
            Hrs_MedAide) / SUM(MDScensus) AS Nurse_Staffing_Ratio
    FROM public.staff_raw
    WHERE MDScensus > 0
    GROUP BY STATE
    ORDER BY Nurse_Staffing_Ratio;

'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('MO', 3.24244682029277)
('IL', 3.25335077395743)
('TX', 3.29513728044523)
('NM', 3.3935734824943)
('WV', 3.45059902152462)
('GA', 3.49111551414975)
('VA', 3.51428807061691)
('NY', 3.59258603543926)
('IN', 3.59604863669311)
('LA', 3.61167990696428)
('NC', 3.62344115256852)
('OH', 3.62784768386199)
('PA', 3.62948860717486)
('RI', 3.632438412123)
('NJ', 3.6355088296775)
('CT', 3.65591613845808)
('SC', 3.65633648041368)
('CO', 3.66615701582845)
('TN', 3.68267344847579)
('OK', 3.70412779289317)
('MA', 3.7241885918198)
('WY', 3.74058578136182)
('NH', 3.74352693604282)
('NV', 3.75693101435567)
('KY', 3.76654574051263)
('MD', 3.76789355974786)
('IA', 3.77195706955568)
('MT', 3.77858736333017)
('SD', 3.79310526086447)
('FL', 3.80468710420148)
('MI', 3.84329569906455)
('AL', 3.85661745115453)
('UT', 3.87309707509974)
('ID', 3.90923994634544)
('KS', 3.96300584063568)
('AZ', 3.98605673695876)
('WI', 3.98714743133763)
('MS', 3.99672239254452)
('VT', 4.06940950218658)
('AR', 4.07101692983774)
('NE'

## 2. Analysis by Employment Type (Contractor, Employer)

### 2.1 Contractor Nurse Staffing Hour Percentage by State

In [3]:
sql_query = '''
    SELECT
        state,
        SUM(
            Hrs_RNDON_ctr +
            Hrs_RNadmin_ctr +
            Hrs_RN_ctr +
            Hrs_LPNadmin_ctr +
            Hrs_LPN_ctr +
            Hrs_CNA_ctr +
            Hrs_NAtrn_ctr +
            Hrs_MedAide_ctr
        ) /
        SUM(
            Hrs_RNDON +
            Hrs_RNadmin +
            Hrs_RN +
            Hrs_LPNadmin +
            Hrs_LPN +
            Hrs_CNA +
            Hrs_NAtrn +
            Hrs_MedAide
        ) AS Contractor_Perc
    FROM
        public.staff_raw
    GROUP BY state
    ORDER BY Contractor_Perc DESC
'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('VT', 0.29639254832869)
('ME', 0.15944246708474)
('NH', 0.153850152724816)
('PA', 0.143562163647583)
('NJ', 0.140117813368113)
('MT', 0.136774715788514)
('DE', 0.130104295299161)
('ND', 0.124996807221619)
('NY', 0.121977730036821)
('OR', 0.112852583964292)
('NE', 0.102984755194681)
('MD', 0.102800731559888)
('AK', 0.102294628104675)
('NC', 0.0994310233862974)
('WI', 0.0982147322135922)
('SD', 0.0915134889806802)
('CO', 0.0906264222615663)
('HI', 0.089118731865989)
('MA', 0.0867348845262703)
('IL', 0.0835444111984028)
('NM', 0.0751747633592423)
('MN', 0.0739634367931947)
('WA', 0.0704015858702044)
('MS', 0.0696476272088264)
('GA', 0.0672017292699964)
('SC', 0.0623464070781175)
('VA', 0.0616381697340084)
('MO', 0.0609103338712016)
('TN', 0.0596673543738582)
('KY', 0.0595010105083592)
('KS', 0.0594518126213861)
('OH', 0.0589253759432898)
('IA', 0.0586304542946027)
('CT', 0.0557474268530435)
('DC', 0.0551335821757881)
('WV', 0.0550982787577767)
('RI', 0.0546255038621399)
('ID', 0.05414746

### 2.2 Contractor Nurse Staffing Hour Percentage by Role

In [4]:
sql_query = '''
    SELECT 
        SUM(Hrs_RNDON_ctr) / NULLIF(SUM(Hrs_RNDON), 0) AS RNDON_ratio,
        SUM(Hrs_RNadmin_ctr) / NULLIF(SUM(Hrs_RNadmin), 0) AS RNadmin_ratio,
        SUM(Hrs_RN_ctr) / NULLIF(SUM(Hrs_RN), 0) AS RN_ratio,
        SUM(Hrs_LPNadmin_ctr) / NULLIF(SUM(Hrs_LPNadmin), 0) AS LPNadmin_ratio,
        SUM(Hrs_LPN_ctr) / NULLIF(SUM(Hrs_LPN), 0) AS LPN_ratio,
        SUM(Hrs_CNA_ctr) / NULLIF(SUM(Hrs_CNA), 0) AS CNA_ratio,
        SUM(Hrs_NAtrn_ctr) / NULLIF(SUM(Hrs_NAtrn), 0) AS NAtrn_ratio,
        SUM(Hrs_MedAide_ctr) / NULLIF(SUM(Hrs_MedAide), 0) AS MedAide_ratio
    FROM 
        public.staff_raw;

'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

(0.0185867162566199, 0.0244683083769214, 0.0807912052549657, 0.0101134880142207, 0.0926887447772934, 0.0700928945543245, 0.0120737770313285, 0.0250658689119716)


In [5]:
sql_query = '''
    SELECT 
        SUM(Hrs_RNDON_ctr) / NULLIF(SUM(Hrs_RNDON), 0) AS RNDON_ratio,
        SUM(Hrs_RNadmin_ctr) / NULLIF(SUM(Hrs_RNadmin), 0) AS RNadmin_ratio,
        SUM(Hrs_RN_ctr) / NULLIF(SUM(Hrs_RN), 0) AS RN_ratio,
        SUM(Hrs_LPNadmin_ctr) / NULLIF(SUM(Hrs_LPNadmin), 0) AS LPNadmin_ratio,
        SUM(Hrs_LPN_ctr) / NULLIF(SUM(Hrs_LPN), 0) AS LPN_ratio,
        SUM(Hrs_CNA_ctr) / NULLIF(SUM(Hrs_CNA), 0) AS CNA_ratio,
        SUM(Hrs_NAtrn_ctr) / NULLIF(SUM(Hrs_NAtrn), 0) AS NAtrn_ratio,
        SUM(Hrs_MedAide_ctr) / NULLIF(SUM(Hrs_MedAide), 0) AS MedAide_ratio
    FROM 
        public.staff_raw;

'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

(0.0185867162566199, 0.0244683083769214, 0.0807912052549657, 0.0101134880142207, 0.0926887447772934, 0.0700928945543245, 0.0120737770313285, 0.0250658689119716)


LPN, RN, CNA

### 2.3 Contractor Nurse Staffing Hour Percentage by Role and State

In [6]:
sql_query = '''
    SELECT 
        state,
        SUM(Hrs_RNDON_ctr) / NULLIF(SUM(Hrs_RNDON), 0) AS RNDON_ratio,
        SUM(Hrs_RNadmin_ctr) / NULLIF(SUM(Hrs_RNadmin), 0) AS RNadmin_ratio,
        SUM(Hrs_RN_ctr) / NULLIF(SUM(Hrs_RN), 0) AS RN_ratio,
        SUM(Hrs_LPNadmin_ctr) / NULLIF(SUM(Hrs_LPNadmin), 0) AS LPNadmin_ratio,
        SUM(Hrs_LPN_ctr) / NULLIF(SUM(Hrs_LPN), 0) AS LPN_ratio,
        SUM(Hrs_CNA_ctr) / NULLIF(SUM(Hrs_CNA), 0) AS CNA_ratio,
        SUM(Hrs_NAtrn_ctr) / NULLIF(SUM(Hrs_NAtrn), 0) AS NAtrn_ratio,
        SUM(Hrs_MedAide_ctr) / NULLIF(SUM(Hrs_MedAide), 0) AS MedAide_ratio
    FROM 
        public.staff_raw
    GROUP BY
        state
'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('NC', 0.0194486068903132, 0.0176957513888144, 0.0929051180093136, 0.00671597850468934, 0.148347058343544, 0.098015855447832, 0.0115884021605778, 0.0446454305900264)
('NY', 0.00403348366896686, 0.0470469929508993, 0.166227465631268, 0.0306511157267844, 0.180791188178905, 0.0981707907404322, 0.120650470123154, 0.140537468063662)
('OH', 0.0112475321266082, 0.0532947783996562, 0.0595336224507991, 0.0172117170348343, 0.0694981563109913, 0.0602368627744775, 2.5491643235598e-05, 0.0113845595018984)
('IA', 0.0123941964545702, 0.0247030673484297, 0.0797967910150252, 0.00484657103920086, 0.0686899556705365, 0.0638938301558257, 0.000510061150258672, 0.0101194348832937)
('KS', 0.0589394175328628, 0.00612209702407777, 0.077280452384632, 0.00587268002936452, 0.0613699711858307, 0.0673391811988666, 0.0, 0.0355179246845979)
('CA', 0.0143247395332502, 0.0247901016530014, 0.023919428936436, 0.00513395506920986, 0.0276455499404159, 0.0417012577475912, 0.000115105558660664, 0.0275506380727778)
('KY', 0.0

## 3. Analysis by Time

### 3.1 The month with the highest MDScensus

In [7]:
sql_query = '''
    SELECT 
        EXTRACT(MONTH FROM WorkDate) AS Month,
        SUM(MDScensus) AS Total_Census
    FROM 
        public.staff_raw
    GROUP BY
        Month
    ORDER BY
        Total_Census DESC
'''
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

(5, 37648478)
(6, 36495133)
(4, 36410544)


### 3.2 The weekday with the highest MDScensus

In [8]:
sql_query = '''
    SELECT 
        TO_CHAR(WorkDate, 'Day') AS Weekday,
        SUM(MDScensus) AS Total_Census
    FROM 
        public.staff_raw
    GROUP BY
        Weekday
    ORDER BY
        Total_Census DESC
'''

cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)


('Friday   ', 15844447)
('Thursday ', 15832921)
('Wednesday', 15810270)
('Saturday ', 15796466)
('Tuesday  ', 15775398)
('Sunday   ', 15749187)
('Monday   ', 15745466)


### 3.3 Top Month of Nursing Staffing Shortage

In [9]:
sql_query = '''
    SELECT 
        EXTRACT(MONTH FROM WorkDate) AS Month,
        SUM(Hrs_RNDON +
            Hrs_RNadmin +
            Hrs_RN +
            Hrs_LPNadmin +
            Hrs_LPN +
            Hrs_CNA +
            Hrs_NAtrn +
            Hrs_MedAide) / SUM(MDScensus) AS Nurse_Staffing_Ratio
    FROM public.staff_raw
    WHERE MDScensus > 0
    GROUP BY Month
    ORDER BY Nurse_Staffing_Ratio;

'''
connection.rollback()
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

(6, 3.70038572814622)
(4, 3.71456179314433)
(5, 3.72708149875275)


### 3.4 Top Weekday of Nursing Staffing Shortage

In [10]:
sql_query = '''
    SELECT 
        TO_CHAR(WorkDate, 'Day') AS Weekday,
        SUM(Hrs_RNDON +
            Hrs_RNadmin +
            Hrs_RN +
            Hrs_LPNadmin +
            Hrs_LPN +
            Hrs_CNA +
            Hrs_NAtrn +
            Hrs_MedAide) / SUM(MDScensus) AS Nurse_Staffing_Ratio
    FROM public.staff_raw
    WHERE MDScensus > 0
    GROUP BY Weekday
    ORDER BY Nurse_Staffing_Ratio;

'''
connection.rollback()
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('Sunday   ', 3.21595781229853)
('Saturday ', 3.28397082993123)
('Monday   ', 3.81773860868901)
('Friday   ', 3.82326390880037)
('Thursday ', 3.93832520227948)
('Tuesday  ', 3.94733127810786)
('Wednesday', 3.97051325562438)


# Close

In [11]:
if cursor:
    cursor.close()
if connection:
    connection.close()